# Python을 활용한 web scraping 기초

2021.08.12

---

# Introduction

* 초보자 대상
* 기본 구조를 이해하고 활용하기 위한 기초 단계 
* 가장 기본적인 방법을 통해 데이터 가져오기

> (데이터 수집 > 저장) > 정리 > 분석 > 리포트/시각화

# Responsible web scraping

* Terms of Service
* robots.txt
* Open API
* Adequate request rate

![data_pipeline_xkcd](https://imgs.xkcd.com/comics/data_pipeline.png)

# 예제 2: Job-Alio 공공기관정보

공공기관정보: https://job.alio.go.kr/orginfo.do

In [1]:
from selenium import webdriver
import pandas as pd
import time

driver = webdriver.Chrome('./chromedriver')

In [2]:
url = 'https://job.alio.go.kr/orginfo.do'
driver.get(url)

## 브라우저 제어

### Checkbox

In [5]:
checkbox1 = driver.find_element_by_xpath('//*[@id="so01"]')
checkbox1.click() 

### Dropdown

In [7]:
from selenium.webdriver.support.ui import Select
dropdown1 = Select(driver.find_element_by_xpath('//*[@id="jumu01"]'))
dropdown1.select_by_value('A1025')

### Search & Reset

In [10]:
driver.execute_script('javascript:fn_search();')

In [9]:
driver.execute_script('javascript:fn_reset();')

## 리스트 수집 - 방식 1

In [1]:
from selenium import webdriver
import pandas as pd
import time

driver = webdriver.Chrome('./chromedriver')
url = 'https://job.alio.go.kr/orginfo.do'
driver.get(url)

In [ ]:
//*[@id="txt"]/table/tbody/tr[1]/td[2]/a
//*[@id="txt"]/table/tbody/tr[2]/td[2]/a
//*[@id="txt"]/table/tbody/tr[3]/td[2]/a

In [12]:
pub_name = []
pub_gov = []
pub_type = []
pub_homepage = []
pub_url = []

rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')

for r in rows:
    pub_name.append(r.find_element_by_xpath('td[2]').text)
    pub_gov.append(r.find_element_by_xpath('td[3]').text)
    pub_type.append(r.find_element_by_xpath('td[4]').text)
    pub_homepage.append(r.find_element_by_xpath('td[5]').text)
    pub_url.append(r.find_element_by_xpath('td[2]/a').get_attribute('onclick'))


In [14]:
pub_homepage

['http://www.sonosa.or.kr',
 'http://www.apcc21.org',
 'http://www.origin.or.kr',
 'http://www.gokams.or.kr',
 'http://www.posid.or.kr',
 'http://www.pola.or.kr',
 'http://www.poma.or.kr',
 'https://www.ilje.or.kr/',
 'http://www.lemi.or.kr/',
 'http://www.posa.or.kr']

In [15]:
page_number = 1
more_page = True

pub_name = []
pub_gov = []
pub_type = []
pub_homepage = []
pub_url = []

while more_page:
    driver.execute_script(f'goPage({page_number});return false;')
    
    time.sleep(2)
    print(f'processing page {page_number}')
    
    rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    
    for r in rows:
        pub_name.append(r.find_element_by_xpath('td[2]').text)
        pub_gov.append(r.find_element_by_xpath('td[3]').text)
        pub_type.append(r.find_element_by_xpath('td[4]').text)
        pub_homepage.append(r.find_element_by_xpath('td[5]').text)
        pub_url.append(r.find_element_by_xpath('td[2]/a').get_attribute('onclick'))
    
    page_number += 1
    if page_number == 3:
        more_page = False

processing page 1
processing page 2


In [3]:
pub_df = pd.DataFrame(list(zip(pub_name, pub_gov, pub_type, pub_homepage, pub_url)), 
                      columns=['name', 'gov', 'type', 'homepage', 'url'])

In [4]:
pub_df

,name,gov,type,homepage,url
0,(사)남북교류협력지원협회,통일부,기타공공기관,http://www.sonosa.or.kr,javascript:fnDetail('C0847');
1,(재)APEC기후센터,기상청,기타공공기관,http://www.apcc21.org,javascript:fnDetail('C0918');
2,(재)국제원산지정보원,관세청,기타공공기관,http://www.origin.or.kr,javascript:fnDetail('C0916');
3,(재)예술경영지원센터,문화체육관광부,기타공공기관,http://www.gokams.or.kr,javascript:fnDetail('C0454');
4,(재)우체국금융개발원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posid.or.kr,javascript:fnDetail('C0352');
5,(재)우체국물류지원단,과학기술정보통신부,준정부기관(위탁집행형),http://www.pola.or.kr,javascript:fnDetail('C0331');
6,(재)우체국시설관리단,과학기술정보통신부,기타공공기관,http://www.poma.or.kr,javascript:fnDetail('C0368');
7,(재)일제강제동원피해자지원재단,행정안전부,기타공공기관,https://www.ilje.or.kr/,javascript:fnDetail('C0931');
8,(재)축산환경관리원,농림축산식품부,기타공공기관,http://www.lemi.or.kr/,javascript:fnDetail('C1030');
9,(재)한국우편사업진흥원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posa.or.kr,javascript:fnDetail('C0354');


## 리스트 수집 - 방식 2

In [19]:
page_number = 1
pub_list = []

while True:
    driver.execute_script(f'goPage({page_number});return false;')
    time.sleep(2)
    print(f'processing page {page_number}')
    
    table = driver.find_element_by_css_selector('#txt > table > tbody')
    rows = table.find_elements_by_tag_name('tr')
    
    for r in rows:
        pub_list.append({
            'name':r.find_elements_by_css_selector('td')[1].text,
            'gov':r.find_elements_by_css_selector('td')[2].text,
            'type':r.find_elements_by_css_selector('td')[3].text,
            'homepage':r.find_elements_by_css_selector('td')[4].text,
            'url':r.find_element_by_css_selector('td.left > a').get_attribute('onclick')
        })
    page_number += 1
    if page_number == 2:
        break

processing page 1


In [20]:
pub_df = pd.DataFrame(pub_list)

In [21]:
pub_df

,name,gov,type,homepage,url
0,(사)남북교류협력지원협회,통일부,기타공공기관,http://www.sonosa.or.kr,javascript:fnDetail('C0847');
1,(재)APEC기후센터,기상청,기타공공기관,http://www.apcc21.org,javascript:fnDetail('C0918');
2,(재)국제원산지정보원,관세청,기타공공기관,http://www.origin.or.kr,javascript:fnDetail('C0916');
3,(재)예술경영지원센터,문화체육관광부,기타공공기관,http://www.gokams.or.kr,javascript:fnDetail('C0454');
4,(재)우체국금융개발원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posid.or.kr,javascript:fnDetail('C0352');
5,(재)우체국물류지원단,과학기술정보통신부,준정부기관(위탁집행형),http://www.pola.or.kr,javascript:fnDetail('C0331');
6,(재)우체국시설관리단,과학기술정보통신부,기타공공기관,http://www.poma.or.kr,javascript:fnDetail('C0368');
7,(재)일제강제동원피해자지원재단,행정안전부,기타공공기관,https://www.ilje.or.kr/,javascript:fnDetail('C0931');
8,(재)축산환경관리원,농림축산식품부,기타공공기관,http://www.lemi.or.kr/,javascript:fnDetail('C1030');
9,(재)한국우편사업진흥원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posa.or.kr,javascript:fnDetail('C0354');


## 본문 수집 - 방식 1

In [22]:
table = driver.find_element_by_css_selector('#txt > table')

headers = table.find_elements_by_tag_name('th')
contents = table.find_elements_by_tag_name('td')

header_list = []
content_list = []

for h in range(len(headers)):
    header_list.append(headers[h].text)
    content_list.append(contents[h].text)

alio_dict = {}
for n in range(len(header_list)):
    alio_dict[header_list[n]] = content_list[n]


In [23]:
alio_dict

{'기관명': '건축공간연구원',
 '기관소개': '건축과 도시공간에 관한 종합적인 연구를 수행함으로써 국민 복리의 향상과 국가 및 지역사회의 지속가능한 발전에 기여\n홈페이지연결 : http://www.auri.re.kr',
 '주무기관': '국무조정실',
 '설립근거': '정부출연연구기관 등의 설립·운영 및 육성에 관한 법률',
 '주요기능 및 역할': '[정관 제4조(사업)]\n1. 건축과 도시공간의 조성, 이용 및 디자인에 대한 종합적인 조사·연구개발\n2. 건축과 도시공간의 개선을 위한 정책 및 제도에 관한 연구\n3. 장소가치와 공간문화 향상에 관한 연구\n4. 국가·지방자치단체 등의 건축물과 공간환경 관련 계획 및 사업 지원\n5. 건축물과 공간환경 관련 정보체계 구축·운영 및 산업의 육성·지원\n6. 국내·외 유관기관과의 공동연구와 국가, 지방자치단체, 공공기관 및 민간단체 등으로부터의 연구용역 수탁\n7. 연구성과의 출판·배포 및 교육·연수·홍보 등 제반사업\n8. 제1호부터 제7호까지의 부대업무와 그 밖에 연구원의 목적달성을 위하여 필요한 업무',
 '기관연혁': "- 2006. 12. 연구소 설립(안) 국회 본회의 통과\n- 2007. 03. 국토연구원 정관개정(안) 의결(정관 및 운영규정)\n- 2007. 06. 초대 소장 온영태 교수 취임\n- 2007. 08. 건축도시공간연구소 개소식\n- 2009. 12. 제2대 소장 손세관 교수 취임\n- 2011. 05. 국가한옥센터 설립('15.7. 지정)\n- 2012. 09. 제3대 소장 제해성 교수 취임\n- 2012. 12. 건축·도시정책정보센터 설립\n- 2013. 06. 녹색건축센터 지정\n- 2014. 02. 도시재생지원기구 지정\n- 2014. 03. 국가공공건축지원센터 설립('14.6. 지정)\n- 2015. 01. 건축서비스산업지원센터 설립\n- 2015. 08. 세종특별자치시 신청사 이전\n- 2015. 09. 제4대 소장 김대익 교수 취임\n- 2017. 12. 개소 10주

In [28]:
driver.execute_script("javascript:fnDetail('C0368');")

In [24]:
pub_contents = []

for u in pub_df['url']:
    driver.execute_script(u)
    time.sleep(2)
    
    table = driver.find_element_by_css_selector('#txt > table')
    headers = table.find_elements_by_tag_name('th')
    contents = table.find_elements_by_tag_name('td')
    header_list = []
    content_list = []
    
    for h in range(len(headers)):
        header_list.append(headers[h].text)
        content_list.append(contents[h].text)
        
    alio_dict = {}
    
    for n in range(len(header_list)):
        alio_dict[header_list[n]] = content_list[n]
        
    pub_contents.append(alio_dict)
    
    driver.back()
    
    time.sleep(2)

In [29]:
pub_contents

[{'기관명': '(사)남북교류협력지원협회',
  '기관소개': '정부 위탁업무 수행, 정책건의, 남북교류협력 관련 조사·연구 및 분석 등을 통한 남북교류협력 활성화 지원\n홈페이지연결 : http://www.sonosa.or.kr',
  '주무기관': '통일부',
  '설립근거': '민법 32조',
  '주요기능 및 역할': 'o 남북교류협력과 관련한 정부 위탁사업 수행\no 정부에 대한 남북교류협력 활성화 대책 건의\no 남북교류협력과 관련한 조사·연구 및 분석\no 남북교류협력 사업자간 정보공유 및 네트워크 구축\no 기타 남북교류협력 촉진을 위한 사업 등',
  '기관연혁': 'o 2007.05.22 법인설립 인가(통일부)\no 2007.05.28「남북 경공업 및 지하자원개발 협력사업」수탁\no 2007.12.05「6자회담 대북 에너지 설비·자재 제공사업」수탁\no 2008.05.23「남북 군사당군간 통신체계 개선사업」수탁\no 2009.09.29「남북교역·경협 관리업무」수탁\no 2011.03.10「남북관리구역 통행체계 보강사업」수탁\no 2012.01.31 기타공공기관 지정(기획재정부)\no 2015.07.02「대북지원사업 통합관리체계 구축·운영 업무」수탁\no 2018.11.05「한강하구 공동조사사업」수탁\no 2018.11.20「남북 산림협력사업 관련 업무」수탁\no 2018.11.22「남북공동 유해발굴 자재장비 지원사업」수탁',
  '경영목표 및 전략': 'o 남북교류협력 활력 제고 및 성장동력 확보\n- 남북교류협력 원스톱 서비스 체계 강화\n- 대국민 정보제공 강화\n- 업무영역 개척 및 기능 확대\no 남북교류협력 추진기반 확대·강화\n- 교류협력 추동력 창출을 위한 환경 조성\n- 교류협력 정책개발 지원 강화\n- 남북교류협력 네트워크 확대·강화\no 경영혁신을 통한 사회적 가치 실현\n- 기관 법적기반 및 조직 역량 강화\n- 사회적 책임에 대한 이행 강화\n- 인권·윤리 및 안전관리 체계 운영',
  '기관장': '강영식',
  '소

In [32]:
pub_df

,name,gov,type,homepage,url
0,(사)남북교류협력지원협회,통일부,기타공공기관,http://www.sonosa.or.kr,javascript:fnDetail('C0847');
1,(재)APEC기후센터,기상청,기타공공기관,http://www.apcc21.org,javascript:fnDetail('C0918');
2,(재)국제원산지정보원,관세청,기타공공기관,http://www.origin.or.kr,javascript:fnDetail('C0916');
3,(재)예술경영지원센터,문화체육관광부,기타공공기관,http://www.gokams.or.kr,javascript:fnDetail('C0454');
4,(재)우체국금융개발원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posid.or.kr,javascript:fnDetail('C0352');
5,(재)우체국물류지원단,과학기술정보통신부,준정부기관(위탁집행형),http://www.pola.or.kr,javascript:fnDetail('C0331');
6,(재)우체국시설관리단,과학기술정보통신부,기타공공기관,http://www.poma.or.kr,javascript:fnDetail('C0368');
7,(재)일제강제동원피해자지원재단,행정안전부,기타공공기관,https://www.ilje.or.kr/,javascript:fnDetail('C0931');
8,(재)축산환경관리원,농림축산식품부,기타공공기관,http://www.lemi.or.kr/,javascript:fnDetail('C1030');
9,(재)한국우편사업진흥원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posa.or.kr,javascript:fnDetail('C0354');


In [30]:
pub_content_df = pd.DataFrame(pub_contents)

In [31]:
pub_content_df

,기관명,기관소개,주무기관,설립근거,주요기능 및 역할,기관연혁,경영목표 및 전략,기관장,소재지
0,(사)남북교류협력지원협회,"정부 위탁업무 수행, 정책건의, 남북교류협력 관련 조사·연구 및 분석 등을 통한 남...",통일부,민법 32조,o 남북교류협력과 관련한 정부 위탁사업 수행\no 정부에 대한 남북교류협력 활성화 ...,o 2007.05.22 법인설립 인가(통일부)\no 2007.05.28「남북 경공업...,o 남북교류협력 활력 제고 및 성장동력 확보\n- 남북교류협력 원스톱 서비스 체계 ...,강영식,"중구 퇴계로 97, 601호"
1,(재)APEC기후센터,기후예측과 그 관련 연구를 통해 기후변화 및 변동의 영향에 효과적으로 대응할 수 있...,기상청,"민법 제32조, 공익법인의 설립·운영에 관한 법률","○ 기후변동 및 변화의 진단, 예측 관련 정보의 수집, 생산 및 유통\n○ 기후변동...",○ 2004.03 제4차 APEC 과학기술장관회의에서 설립 제안\n○ 2005.03...,○ 기후예측 전문성 강화 및 국내 기여 확대\n○ 기후예측 기술 개발 및 기후 감시...,권원태,해운대구 센텀7로 12
2,(재)국제원산지정보원,자유무역협정 및 원산지 관련 연구ㆍ조사ㆍ정보 보급 확대 등을 통하여 우리 기업의<b...,관세청,민법 제32조(비영리법인의 설립과 허가),1. 자유무역협정ㆍ원산지 정보의 수집ㆍ분석 수탁 사업\n2. 자유무역협정ㆍ원산지 정...,2009. 1 원산지정보 수집· 분석 전문기관 지정(관세청)\n2009. 2 「한국...,미션 : FTA 및 원산지 연구· 조사와 정보보급을 통해 기업경쟁력 강화와 국민경제...,박병진,성남시 분당구 야탑로 205번길8
3,(재)예술경영지원센터,"예술유통의 활성화와 예술기관의 경쟁력 강화를 종합적이고 체계적으로 지원함으로써, 예...",문화체육관광부,민법 제32조에 의한 재단법인,1. 예술 유통구조의 체계화 및 활성화 지원\n2. 예술기관·단체 운영 및 경영 관...,"○ 2006년 재단법인 예술경영지원센터 설립 (2006.1.12.), 제1기 이사회...",○ 비전 : 예술시장 활성화로 예술현장의 지속성장을 이끌어가는 기관\n○ 핵심가치 ...,김도일,"종로구 대학로 57(연건동) 홍익대학교 대학로캠퍼스 교육동 3층, 12층"
4,(재)우체국금융개발원,우체국금융 업무를 효과적으로 지원함으로써 우체국예금보험사업의 향상·발전에 기여\n홈...,과학기술정보통신부,"민법 제32조(비영리법인의 설립과 허가) 학술, 종교, 자선, 기예, 사교 기타 영...",ㅇ 국내외 금융시장에 대한 조사·연구\nㅇ 우체국금융 신서비스 도입 및 마케팅 방안...,1966.04. 재단법인 체신저축장려회 설립\n1976.12. 재단법인 체신장려회로...,◎ 미션 : 안전하고 편리한 금융서비스를 통해 국민의 삶의 질 향상에 기여한다\n\...,유대선,영등포구 경인로 841(영등포동4가)
5,(재)우체국물류지원단,안전ㆍ신속ㆍ정확한 우편물류서비스를 효율적으로 제공함으로써 국민의 편익 <br />증...,과학기술정보통신부,민법 제32조,"'우체국물류지원단'은 전국 우편물 운송, 배달, 분류, 국제우편물 포워딩 등 종합우...",□ 1980.08.26 법인설립 (재)체신복지회\n□ 1981.03.01 수도권 우...,□ (기관미션) 안전ㆍ 신속ㆍ 정확한 우편물류서비스를 효율적으로 제공함으로써\n국민...,천장수,광진구 자양로 76
6,(재)우체국시설관리단,우정사업조직에 속한 부동산의 효율적 관리ㆍ운영으로 우정자산의 적극적 활용 및 가치향...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),o 우정재산의 사용ㆍ수익 허가 및 대부업무 지원사업\no 우정사업 부동산 시설 유지...,o 2001. 11. 23. 법인 설립\no 2006. 12. 28. 우체국사 관리...,⊙ 미션 : 전문적이고 효율적인 우정시설 및 자산관리를 통해 공공의 복리 증진에 기...,김성칠,"광진구 강벽역로 2(구의동, 우체국시설관리단)"
7,(재)일제강제동원피해자지원재단,"일제강제동원 피해·희생자 및 유족 등에 대한 복지지원사업, 추념사업 및 강제동원 피...",행정안전부,"- 설립근거「민법」제32조(비영리 법인의 설립과 허가),「공익법인의 설립·운영에 관...",- 일제 강제동원 희생자에 대한 추도순례 등 위령사업- 해외 추도공간 조성 및 일제...,- 2014. 6. 2. / 3. 재단 설립허가/등기- 2015. 1. 29. 기타...,"- 미션: 과거의 기억, 현재의 통합, 미래의 평화- 비젼: 국민과 함께 평화와 인...",김용덕,"종로구 종로1길 42, 603호(수송동, 이마빌딩)"
8,(재)축산환경관리원,친환경적인 가축사육환경 조성 및 가축분뇨의 자원화를 효율적으로 수행함으로써<br /...,농림축산식품부,「가축분뇨의 관리 및 이용에 관한 법률」제38조의2(축산환경관리원의 설립·운영),"o 배출시설 또는 처리시설 컨설팅, 지도 및 교육, 퇴비·액비의 품질관리\no 가축...","o 민간관리기구 도입검토('10.7월, 국무총리실 주관 T/F팀)o '가축분뇨의 효...",o (비전) 국민에게 신뢰받는 세계 최고의 축산환경개선 선도기관\no (미션) 지속...,이영희,"한누리대로 219, 7층(나성동, 한림프라자)"
9,(재)한국우편사업진흥원,우편서비스의 질적 향상 및 우정문화를 창달함으로써 우정사업 발전과 국민문화 생활에 ...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),1. 문화우편상품과 우표류 보급 및 콘텐츠를 통한 우정문화 확산\n2. 우정 및 그...,ㅇ 1930. 12. 03 우편소청사협회 설립\nㅇ 1939. 11. 06 조선체신...,ㅇ 미션: 우정문화의 가치혁신과 선진 우편서비스의 제공으로 국민의 행복과 공익실현에...,민재석,영등포구 영중로 83


In [33]:
pub_df_merge = pd.merge(pub_df, pub_content_df, left_on='name', right_on='기관명', how='left')

In [34]:
pub_df_merge

,name,gov,type,homepage,url,기관명,기관소개,주무기관,설립근거,주요기능 및 역할,기관연혁,경영목표 및 전략,기관장,소재지
0,(사)남북교류협력지원협회,통일부,기타공공기관,http://www.sonosa.or.kr,javascript:fnDetail('C0847');,(사)남북교류협력지원협회,"정부 위탁업무 수행, 정책건의, 남북교류협력 관련 조사·연구 및 분석 등을 통한 남...",통일부,민법 32조,o 남북교류협력과 관련한 정부 위탁사업 수행\no 정부에 대한 남북교류협력 활성화 ...,o 2007.05.22 법인설립 인가(통일부)\no 2007.05.28「남북 경공업...,o 남북교류협력 활력 제고 및 성장동력 확보\n- 남북교류협력 원스톱 서비스 체계 ...,강영식,"중구 퇴계로 97, 601호"
1,(재)APEC기후센터,기상청,기타공공기관,http://www.apcc21.org,javascript:fnDetail('C0918');,(재)APEC기후센터,기후예측과 그 관련 연구를 통해 기후변화 및 변동의 영향에 효과적으로 대응할 수 있...,기상청,"민법 제32조, 공익법인의 설립·운영에 관한 법률","○ 기후변동 및 변화의 진단, 예측 관련 정보의 수집, 생산 및 유통\n○ 기후변동...",○ 2004.03 제4차 APEC 과학기술장관회의에서 설립 제안\n○ 2005.03...,○ 기후예측 전문성 강화 및 국내 기여 확대\n○ 기후예측 기술 개발 및 기후 감시...,권원태,해운대구 센텀7로 12
2,(재)국제원산지정보원,관세청,기타공공기관,http://www.origin.or.kr,javascript:fnDetail('C0916');,(재)국제원산지정보원,자유무역협정 및 원산지 관련 연구ㆍ조사ㆍ정보 보급 확대 등을 통하여 우리 기업의<b...,관세청,민법 제32조(비영리법인의 설립과 허가),1. 자유무역협정ㆍ원산지 정보의 수집ㆍ분석 수탁 사업\n2. 자유무역협정ㆍ원산지 정...,2009. 1 원산지정보 수집· 분석 전문기관 지정(관세청)\n2009. 2 「한국...,미션 : FTA 및 원산지 연구· 조사와 정보보급을 통해 기업경쟁력 강화와 국민경제...,박병진,성남시 분당구 야탑로 205번길8
3,(재)예술경영지원센터,문화체육관광부,기타공공기관,http://www.gokams.or.kr,javascript:fnDetail('C0454');,(재)예술경영지원센터,"예술유통의 활성화와 예술기관의 경쟁력 강화를 종합적이고 체계적으로 지원함으로써, 예...",문화체육관광부,민법 제32조에 의한 재단법인,1. 예술 유통구조의 체계화 및 활성화 지원\n2. 예술기관·단체 운영 및 경영 관...,"○ 2006년 재단법인 예술경영지원센터 설립 (2006.1.12.), 제1기 이사회...",○ 비전 : 예술시장 활성화로 예술현장의 지속성장을 이끌어가는 기관\n○ 핵심가치 ...,김도일,"종로구 대학로 57(연건동) 홍익대학교 대학로캠퍼스 교육동 3층, 12층"
4,(재)우체국금융개발원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posid.or.kr,javascript:fnDetail('C0352');,(재)우체국금융개발원,우체국금융 업무를 효과적으로 지원함으로써 우체국예금보험사업의 향상·발전에 기여\n홈...,과학기술정보통신부,"민법 제32조(비영리법인의 설립과 허가) 학술, 종교, 자선, 기예, 사교 기타 영...",ㅇ 국내외 금융시장에 대한 조사·연구\nㅇ 우체국금융 신서비스 도입 및 마케팅 방안...,1966.04. 재단법인 체신저축장려회 설립\n1976.12. 재단법인 체신장려회로...,◎ 미션 : 안전하고 편리한 금융서비스를 통해 국민의 삶의 질 향상에 기여한다\n\...,유대선,영등포구 경인로 841(영등포동4가)
5,(재)우체국물류지원단,과학기술정보통신부,준정부기관(위탁집행형),http://www.pola.or.kr,javascript:fnDetail('C0331');,(재)우체국물류지원단,안전ㆍ신속ㆍ정확한 우편물류서비스를 효율적으로 제공함으로써 국민의 편익 <br />증...,과학기술정보통신부,민법 제32조,"'우체국물류지원단'은 전국 우편물 운송, 배달, 분류, 국제우편물 포워딩 등 종합우...",□ 1980.08.26 법인설립 (재)체신복지회\n□ 1981.03.01 수도권 우...,□ (기관미션) 안전ㆍ 신속ㆍ 정확한 우편물류서비스를 효율적으로 제공함으로써\n국민...,천장수,광진구 자양로 76
6,(재)우체국시설관리단,과학기술정보통신부,기타공공기관,http://www.poma.or.kr,javascript:fnDetail('C0368');,(재)우체국시설관리단,우정사업조직에 속한 부동산의 효율적 관리ㆍ운영으로 우정자산의 적극적 활용 및 가치향...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),o 우정재산의 사용ㆍ수익 허가 및 대부업무 지원사업\no 우정사업 부동산 시설 유지...,o 2001. 11. 23. 법인 설립\no 2006. 12. 28. 우체국사 관리...,⊙ 미션 : 전문적이고 효율적인 우정시설 및 자산관리를 통해 공공의 복리 증진에 기...,김성칠,"광진구 강벽역로 2(구의동, 우체국시설관리단)"
7,(재)일제강제동원피해자지원재단,행정안전부,기타공공기관,https://www.ilje.or.kr/,javascript:fnDetail('C0931');,(재)일제강제동원피해자지원재단,"일제강제동원 피해·희생자 및 유족 등에 대한 복지지원사업, 추념사업 및 강제동원 피...",행정안전부,"- 설립근거「민법」제32조(비영리 법인의 설립과 허가),「공익법인의 설립·운영에 관...",- 일제 강제동원 희생자에 대한 추도순례 등 위령사업- 해외 추도공간 조성 및 일제...,- 2014. 6. 2. / 3. 재단 설립허가/등기- 2015. 1. 29. 기타...,"- 미션: 과거의 기억, 현재의 통합, 미래의 평화- 비젼: 국민과 함께 평화와 인...",김용덕,"종로구 종로1길 42, 603호(수송동, 이마빌딩)"
8,(재)축산환경관리원,농림축산식품부,기타공공기관,http://www.lemi.or.kr/,javascript:fnDetail('C1030');,(재)축산환경관리원,친환경적인 가축사육환경 조성 및 가축분뇨의 자원화를 효율적으로 수행함으로써<br /...,농림축산식품부,「가축분뇨의 관리 및 이용에 관한 법률」제38조의2(축산환경관리원의 설립·운영),"o 배출시설 또는 처리시설 컨설팅, 지도 및 교육, 퇴비·액비의 품질관리\no 가축...","o 민간관리기구 도입검토('10.7월, 국무총리실 주관 T/F팀)o '가축분뇨의 효...",o (비전) 국민에게 신뢰받는 세계 최고의 축산환경개선 선도기관\no (미션) 지속...,이영희,"한누리대로 219, 7층(나성동, 한림프라자)"
9,(재)한국우편사업진흥원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posa.or.kr,javascript:fnDetail('C0354');,(재)한국우편사업진흥원,우편서비스의 질적 향상 및 우정문화를 창달함으로써 우정사업 발전과 국민문화 생활에 ...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),1. 문화우편상품과 우표류 보급 및 콘텐츠를 통한 우정문화 확산\n2. 우정 및 그...,ㅇ 1930. 12. 03 우편소청사협회 설립\nㅇ 1939. 11. 06 조선체신...,ㅇ 미션: 우정문화의 가치혁신과 선진 우편서비스의 제공으로 국민의 행복과 공익실현에...,민재석,영등포구 영중로 83


In [35]:
pub_df_concat = pd.concat([pub_df, pub_content_df], axis=1)

In [36]:
pub_df_concat

,name,gov,type,homepage,url,기관명,기관소개,주무기관,설립근거,주요기능 및 역할,기관연혁,경영목표 및 전략,기관장,소재지
0,(사)남북교류협력지원협회,통일부,기타공공기관,http://www.sonosa.or.kr,javascript:fnDetail('C0847');,(사)남북교류협력지원협회,"정부 위탁업무 수행, 정책건의, 남북교류협력 관련 조사·연구 및 분석 등을 통한 남...",통일부,민법 32조,o 남북교류협력과 관련한 정부 위탁사업 수행\no 정부에 대한 남북교류협력 활성화 ...,o 2007.05.22 법인설립 인가(통일부)\no 2007.05.28「남북 경공업...,o 남북교류협력 활력 제고 및 성장동력 확보\n- 남북교류협력 원스톱 서비스 체계 ...,강영식,"중구 퇴계로 97, 601호"
1,(재)APEC기후센터,기상청,기타공공기관,http://www.apcc21.org,javascript:fnDetail('C0918');,(재)APEC기후센터,기후예측과 그 관련 연구를 통해 기후변화 및 변동의 영향에 효과적으로 대응할 수 있...,기상청,"민법 제32조, 공익법인의 설립·운영에 관한 법률","○ 기후변동 및 변화의 진단, 예측 관련 정보의 수집, 생산 및 유통\n○ 기후변동...",○ 2004.03 제4차 APEC 과학기술장관회의에서 설립 제안\n○ 2005.03...,○ 기후예측 전문성 강화 및 국내 기여 확대\n○ 기후예측 기술 개발 및 기후 감시...,권원태,해운대구 센텀7로 12
2,(재)국제원산지정보원,관세청,기타공공기관,http://www.origin.or.kr,javascript:fnDetail('C0916');,(재)국제원산지정보원,자유무역협정 및 원산지 관련 연구ㆍ조사ㆍ정보 보급 확대 등을 통하여 우리 기업의<b...,관세청,민법 제32조(비영리법인의 설립과 허가),1. 자유무역협정ㆍ원산지 정보의 수집ㆍ분석 수탁 사업\n2. 자유무역협정ㆍ원산지 정...,2009. 1 원산지정보 수집· 분석 전문기관 지정(관세청)\n2009. 2 「한국...,미션 : FTA 및 원산지 연구· 조사와 정보보급을 통해 기업경쟁력 강화와 국민경제...,박병진,성남시 분당구 야탑로 205번길8
3,(재)예술경영지원센터,문화체육관광부,기타공공기관,http://www.gokams.or.kr,javascript:fnDetail('C0454');,(재)예술경영지원센터,"예술유통의 활성화와 예술기관의 경쟁력 강화를 종합적이고 체계적으로 지원함으로써, 예...",문화체육관광부,민법 제32조에 의한 재단법인,1. 예술 유통구조의 체계화 및 활성화 지원\n2. 예술기관·단체 운영 및 경영 관...,"○ 2006년 재단법인 예술경영지원센터 설립 (2006.1.12.), 제1기 이사회...",○ 비전 : 예술시장 활성화로 예술현장의 지속성장을 이끌어가는 기관\n○ 핵심가치 ...,김도일,"종로구 대학로 57(연건동) 홍익대학교 대학로캠퍼스 교육동 3층, 12층"
4,(재)우체국금융개발원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posid.or.kr,javascript:fnDetail('C0352');,(재)우체국금융개발원,우체국금융 업무를 효과적으로 지원함으로써 우체국예금보험사업의 향상·발전에 기여\n홈...,과학기술정보통신부,"민법 제32조(비영리법인의 설립과 허가) 학술, 종교, 자선, 기예, 사교 기타 영...",ㅇ 국내외 금융시장에 대한 조사·연구\nㅇ 우체국금융 신서비스 도입 및 마케팅 방안...,1966.04. 재단법인 체신저축장려회 설립\n1976.12. 재단법인 체신장려회로...,◎ 미션 : 안전하고 편리한 금융서비스를 통해 국민의 삶의 질 향상에 기여한다\n\...,유대선,영등포구 경인로 841(영등포동4가)
5,(재)우체국물류지원단,과학기술정보통신부,준정부기관(위탁집행형),http://www.pola.or.kr,javascript:fnDetail('C0331');,(재)우체국물류지원단,안전ㆍ신속ㆍ정확한 우편물류서비스를 효율적으로 제공함으로써 국민의 편익 <br />증...,과학기술정보통신부,민법 제32조,"'우체국물류지원단'은 전국 우편물 운송, 배달, 분류, 국제우편물 포워딩 등 종합우...",□ 1980.08.26 법인설립 (재)체신복지회\n□ 1981.03.01 수도권 우...,□ (기관미션) 안전ㆍ 신속ㆍ 정확한 우편물류서비스를 효율적으로 제공함으로써\n국민...,천장수,광진구 자양로 76
6,(재)우체국시설관리단,과학기술정보통신부,기타공공기관,http://www.poma.or.kr,javascript:fnDetail('C0368');,(재)우체국시설관리단,우정사업조직에 속한 부동산의 효율적 관리ㆍ운영으로 우정자산의 적극적 활용 및 가치향...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),o 우정재산의 사용ㆍ수익 허가 및 대부업무 지원사업\no 우정사업 부동산 시설 유지...,o 2001. 11. 23. 법인 설립\no 2006. 12. 28. 우체국사 관리...,⊙ 미션 : 전문적이고 효율적인 우정시설 및 자산관리를 통해 공공의 복리 증진에 기...,김성칠,"광진구 강벽역로 2(구의동, 우체국시설관리단)"
7,(재)일제강제동원피해자지원재단,행정안전부,기타공공기관,https://www.ilje.or.kr/,javascript:fnDetail('C0931');,(재)일제강제동원피해자지원재단,"일제강제동원 피해·희생자 및 유족 등에 대한 복지지원사업, 추념사업 및 강제동원 피...",행정안전부,"- 설립근거「민법」제32조(비영리 법인의 설립과 허가),「공익법인의 설립·운영에 관...",- 일제 강제동원 희생자에 대한 추도순례 등 위령사업- 해외 추도공간 조성 및 일제...,- 2014. 6. 2. / 3. 재단 설립허가/등기- 2015. 1. 29. 기타...,"- 미션: 과거의 기억, 현재의 통합, 미래의 평화- 비젼: 국민과 함께 평화와 인...",김용덕,"종로구 종로1길 42, 603호(수송동, 이마빌딩)"
8,(재)축산환경관리원,농림축산식품부,기타공공기관,http://www.lemi.or.kr/,javascript:fnDetail('C1030');,(재)축산환경관리원,친환경적인 가축사육환경 조성 및 가축분뇨의 자원화를 효율적으로 수행함으로써<br /...,농림축산식품부,「가축분뇨의 관리 및 이용에 관한 법률」제38조의2(축산환경관리원의 설립·운영),"o 배출시설 또는 처리시설 컨설팅, 지도 및 교육, 퇴비·액비의 품질관리\no 가축...","o 민간관리기구 도입검토('10.7월, 국무총리실 주관 T/F팀)o '가축분뇨의 효...",o (비전) 국민에게 신뢰받는 세계 최고의 축산환경개선 선도기관\no (미션) 지속...,이영희,"한누리대로 219, 7층(나성동, 한림프라자)"
9,(재)한국우편사업진흥원,과학기술정보통신부,준정부기관(위탁집행형),http://www.posa.or.kr,javascript:fnDetail('C0354');,(재)한국우편사업진흥원,우편서비스의 질적 향상 및 우정문화를 창달함으로써 우정사업 발전과 국민문화 생활에 ...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),1. 문화우편상품과 우표류 보급 및 콘텐츠를 통한 우정문화 확산\n2. 우정 및 그...,ㅇ 1930. 12. 03 우편소청사협회 설립\nㅇ 1939. 11. 06 조선체신...,ㅇ 미션: 우정문화의 가치혁신과 선진 우편서비스의 제공으로 국민의 행복과 공익실현에...,민재석,영등포구 영중로 83


## 본문 수집 - 방식 2

### url 구성에 필요한 정보 추출

In [ ]:
# 'https://job.alio.go.kr/orginfoview.do?apba_id=C0005'

# https://job.alio.go.kr/orginfoview.do?pageNo=1&apba_id=C0847

In [37]:
pub_url[0]

"javascript:fnDetail('C0847');"

In [39]:
apba_id = pub_url[0].split("'")

In [40]:
apba_id

['javascript:fnDetail(', 'C0847', ');']

In [42]:
apba_id = pub_url[0].split("'")[1]
print(apba_id)

C0847


In [43]:
url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id
print(url)

https://job.alio.go.kr/orginfoview.do?apba_id=C0847


In [44]:
pub_real_urls = []
for n in pub_url:
    apba_id = n.split("'")[1]
    pub_real_urls.append('https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id)

In [46]:
pub_real_urls

['https://job.alio.go.kr/orginfoview.do?apba_id=C0847',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0918',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0916',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0454',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0352',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0331',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0368',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0931',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C1030',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0354',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0005',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C1024',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0840',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0125',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0001',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0002',
 'https://job.alio.go.kr/orginfoview.do?apba_id=C0003',
 'https://job.alio.go.kr/orginfoview.do?apba_id=

In [47]:
pub_contents = []
for u in pub_real_urls:
    driver.get(u)
    time.sleep(2)
    
    rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    alio_dict = {}
    for r in rows:
        header = r.find_element_by_tag_name('th').text
        content = r.find_element_by_tag_name('td').text
        alio_dict[header]=content
        
    pub_contents.append(alio_dict)

In [ ]:
pub_content_df = pd.DataFrame(pub_contents)
pub_content_df

### 정리 1

In [5]:
pub_contents = []

for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id
    
    driver.get(pub_real_url)
    time.sleep(2)
    
    rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    
    alio_dict = {}
    for r in rows:
        header = r.find_element_by_tag_name('th').text
        content = r.find_element_by_tag_name('td').text
        alio_dict[header]=content
        
    pub_contents.append(alio_dict)

In [6]:
pub_content_df = pd.DataFrame(pub_contents)
pub_content_df

,기관명,기관소개,주무기관,설립근거,주요기능 및 역할,기관연혁,경영목표 및 전략,기관장,소재지
0,(사)남북교류협력지원협회,"정부 위탁업무 수행, 정책건의, 남북교류협력 관련 조사·연구 및 분석 등을 통한 남...",통일부,민법 32조,o 남북교류협력과 관련한 정부 위탁사업 수행\no 정부에 대한 남북교류협력 활성화 ...,o 2007.05.22 법인설립 인가(통일부)\no 2007.05.28「남북 경공업...,o 남북교류협력 활력 제고 및 성장동력 확보\n- 남북교류협력 원스톱 서비스 체계 ...,강영식,"중구 퇴계로 97, 601호"
1,(재)APEC기후센터,기후예측과 그 관련 연구를 통해 기후변화 및 변동의 영향에 효과적으로 대응할 수 있...,기상청,"민법 제32조, 공익법인의 설립·운영에 관한 법률","○ 기후변동 및 변화의 진단, 예측 관련 정보의 수집, 생산 및 유통\n○ 기후변동...",○ 2004.03 제4차 APEC 과학기술장관회의에서 설립 제안\n○ 2005.03...,○ 기후예측 전문성 강화 및 국내 기여 확대\n○ 기후예측 기술 개발 및 기후 감시...,권원태,해운대구 센텀7로 12
2,(재)국제원산지정보원,자유무역협정 및 원산지 관련 연구ㆍ조사ㆍ정보 보급 확대 등을 통하여 우리 기업의<b...,관세청,민법 제32조(비영리법인의 설립과 허가),1. 자유무역협정ㆍ원산지 정보의 수집ㆍ분석 수탁 사업\n2. 자유무역협정ㆍ원산지 정...,2009. 1 원산지정보 수집· 분석 전문기관 지정(관세청)\n2009. 2 「한국...,미션 : FTA 및 원산지 연구· 조사와 정보보급을 통해 기업경쟁력 강화와 국민경제...,박병진,성남시 분당구 야탑로 205번길8
3,(재)예술경영지원센터,"예술유통의 활성화와 예술기관의 경쟁력 강화를 종합적이고 체계적으로 지원함으로써, 예...",문화체육관광부,민법 제32조에 의한 재단법인,1. 예술 유통구조의 체계화 및 활성화 지원\n2. 예술기관·단체 운영 및 경영 관...,"○ 2006년 재단법인 예술경영지원센터 설립 (2006.1.12.), 제1기 이사회...",○ 비전 : 예술시장 활성화로 예술현장의 지속성장을 이끌어가는 기관\n○ 핵심가치 ...,김도일,"종로구 대학로 57(연건동) 홍익대학교 대학로캠퍼스 교육동 3층, 12층"
4,(재)우체국금융개발원,우체국금융 업무를 효과적으로 지원함으로써 우체국예금보험사업의 향상·발전에 기여\n홈...,과학기술정보통신부,"민법 제32조(비영리법인의 설립과 허가) 학술, 종교, 자선, 기예, 사교 기타 영...",ㅇ 국내외 금융시장에 대한 조사·연구\nㅇ 우체국금융 신서비스 도입 및 마케팅 방안...,1966.04. 재단법인 체신저축장려회 설립\n1976.12. 재단법인 체신장려회로...,◎ 미션 : 안전하고 편리한 금융서비스를 통해 국민의 삶의 질 향상에 기여한다\n\...,유대선,영등포구 경인로 841(영등포동4가)
5,(재)우체국물류지원단,안전ㆍ신속ㆍ정확한 우편물류서비스를 효율적으로 제공함으로써 국민의 편익 <br />증...,과학기술정보통신부,민법 제32조,"'우체국물류지원단'은 전국 우편물 운송, 배달, 분류, 국제우편물 포워딩 등 종합우...",□ 1980.08.26 법인설립 (재)체신복지회\n□ 1981.03.01 수도권 우...,□ (기관미션) 안전ㆍ 신속ㆍ 정확한 우편물류서비스를 효율적으로 제공함으로써\n국민...,천장수,광진구 자양로 76
6,(재)우체국시설관리단,우정사업조직에 속한 부동산의 효율적 관리ㆍ운영으로 우정자산의 적극적 활용 및 가치향...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),o 우정재산의 사용ㆍ수익 허가 및 대부업무 지원사업\no 우정사업 부동산 시설 유지...,o 2001. 11. 23. 법인 설립\no 2006. 12. 28. 우체국사 관리...,⊙ 미션 : 전문적이고 효율적인 우정시설 및 자산관리를 통해 공공의 복리 증진에 기...,김성칠,"광진구 강벽역로 2(구의동, 우체국시설관리단)"
7,(재)일제강제동원피해자지원재단,"일제강제동원 피해·희생자 및 유족 등에 대한 복지지원사업, 추념사업 및 강제동원 피...",행정안전부,"- 설립근거「민법」제32조(비영리 법인의 설립과 허가),「공익법인의 설립·운영에 관...",- 일제 강제동원 희생자에 대한 추도순례 등 위령사업- 해외 추도공간 조성 및 일제...,- 2014. 6. 2. / 3. 재단 설립허가/등기- 2015. 1. 29. 기타...,"- 미션: 과거의 기억, 현재의 통합, 미래의 평화- 비젼: 국민과 함께 평화와 인...",김용덕,"종로구 종로1길 42, 603호(수송동, 이마빌딩)"
8,(재)축산환경관리원,친환경적인 가축사육환경 조성 및 가축분뇨의 자원화를 효율적으로 수행함으로써<br /...,농림축산식품부,「가축분뇨의 관리 및 이용에 관한 법률」제38조의2(축산환경관리원의 설립·운영),"o 배출시설 또는 처리시설 컨설팅, 지도 및 교육, 퇴비·액비의 품질관리\no 가축...","o 민간관리기구 도입검토('10.7월, 국무총리실 주관 T/F팀)o '가축분뇨의 효...",o (비전) 국민에게 신뢰받는 세계 최고의 축산환경개선 선도기관\no (미션) 지속...,이영희,"한누리대로 219, 7층(나성동, 한림프라자)"
9,(재)한국우편사업진흥원,우편서비스의 질적 향상 및 우정문화를 창달함으로써 우정사업 발전과 국민문화 생활에 ...,과학기술정보통신부,민법 제32조(비영리법인의 설립과 허가),1. 문화우편상품과 우표류 보급 및 콘텐츠를 통한 우정문화 확산\n2. 우정 및 그...,ㅇ 1930. 12. 03 우편소청사협회 설립\nㅇ 1939. 11. 06 조선체신...,ㅇ 미션: 우정문화의 가치혁신과 선진 우편서비스의 제공으로 국민의 행복과 공익실현에...,민재석,영등포구 영중로 83


### 정리 2

In [48]:
import csv

file_name = "공공기관정보(job-alio).csv"
f = open(file_name, 'w', encoding='utf-8-sig', newline='')
writer = csv.writer(f)

for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id
    
    driver.get(pub_real_url)
    time.sleep(2)
    
    rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    
    content = []
    for r in rows:
        content.append(r.find_element_by_tag_name('td').text)
        
    writer.writerow(content)    
f.close()

In [10]:
import csv

file_name = "공공기관정보(job-alio)-1.csv"
f = open(file_name, 'w', encoding='utf-8-sig', newline='')
writer = csv.writer(f)

for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id
    
    driver.get(pub_real_url)
    time.sleep(2)
    
    rows = driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    
    content = [r.find_element_by_tag_name('td').text for r in rows]
    writer.writerow(content)    
    
f.close()

## Headless Chrome

### User Agents

웹사이트에 접속할 때 어느 브라우저를 쓰는지, OS를 쓰는지, 컴퓨터로 접속하는지, 스마트폰으로 접속하는지 등 정보를 서버가 알 수 있음  
사람이 접속하는 것이 아니라 컴퓨터가 접속하는 경우 사이트에서는 접속을 차단할 수 있음  

https://www.whatismybrowser.com/detect/what-is-my-user-agent

![user_agent_meme](https://memegenerator.net/img/instances/64455057/you-said-your-device-is-windows-phone-but-your-user-agent-string-determined-that-to-be-a-lie.jpg)

In [49]:
headless_options = webdriver.ChromeOptions()
headless_options.headless = True
headless_options.add_argument('window-size=1920x1080')  
headless_options.add_argument('User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36')  
headless_options.add_argument('lang=ko_KR')  
headless_options.add_argument('disable-gpu')  

In [ ]:
# options = webdriver.ChromeOptions()
# options.add_argument('--ignore-ssl-errors=yes')
# options.add_argument('--ignore-certificate-errors')
# driver = webdriver.Chrome(options=options)

In [50]:
from selenium import webdriver
import pandas as pd
import time

headless_driver = webdriver.Chrome('./chromedriver', options=headless_options)
url = 'https://job.alio.go.kr/orginfo.do'
headless_driver.get(url)

In [51]:
headless_driver.get_screenshot_as_file("./img/Job-Alio_first.png")

True

screenshot 

![screenshot_1](./img/Job-Alio_first.png)

In [52]:
checkbox1 = headless_driver.find_element_by_xpath('//*[@id="so01"]')
checkbox1.click()

In [56]:
headless_driver.get_screenshot_as_file("./img/Job-Alio_checkbox.png")

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=52511): Max retries exceeded with url: /session/ea201a41c8097b7288df289e4d0b0e41/screenshot (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002CE27594A60>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

screenshot2 

![screenshot_2](./img/Job-Alio_checkbox.png)

In [54]:
pub_contents = []

for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id

    headless_driver.get(pub_real_url)
    time.sleep(2)
    print(f'processing {apba_id}')
    
    rows = headless_driver.find_elements_by_xpath('//*[@id="txt"]/table/tbody/tr')
    alio_dict = {}
    for r in rows:
        header = r.find_element_by_tag_name('th').text
        content = r.find_element_by_tag_name('td').text
        alio_dict[header]=content
        
    pub_contents.append(alio_dict)

processing C0847
processing C0918
processing C0916
processing C0454
processing C0352
processing C0331
processing C0368
processing C0931
processing C1030
processing C0354
processing C0005
processing C1024
processing C0840
processing C0125
processing C0001
processing C0002
processing C0003
processing C0004
processing C0006
processing C0672


In [ ]:
pub_content_df1 = pd.DataFrame(pub_contents)
pub_content_df1

In [55]:
headless_driver.quit()

## Requests & BeautifulSoup

In [57]:
import requests
from bs4 import BeautifulSoup

pub_contents = []
for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id

    res = requests.get(pub_real_url)
    time.sleep(2)
    print(f'processing {apba_id}')
    
    soup = BeautifulSoup(res.text, 'html.parser')
    rows = soup.select('#txt > table > tbody > tr')
    
    alio_dict = {}
    for r in rows:
        header = r.select_one('th').get_text()
        content = r.select_one('td').get_text()
        alio_dict[header]=content
        
    pub_contents.append(alio_dict)

processing C0847
processing C0918
processing C0916
processing C0454
processing C0352
processing C0331
processing C0368
processing C0931
processing C1030
processing C0354
processing C0005
processing C1024
processing C0840
processing C0125
processing C0001
processing C0002
processing C0003
processing C0004
processing C0006
processing C0672


In [ ]:
pub_content_df_request = pd.DataFrame(pub_contents)
pub_content_df_request

### HTTP response code

HTTP라는 프로토콜 규격에 따라서 응답 데이터에 응답 코드를 함께 받음  
https://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html

응답코드가 200인 경우는 정상 응답

In [ ]:
res = requests.get('https://job.alio.go.kr/orginfoview.do?apba_id=C0847')
print(res.status_code)

In [ ]:
import requests
from bs4 import BeautifulSoup

pub_contents = []
for u in pub_url:
    apba_id = u.split("'")[1]
    pub_real_url = 'https://job.alio.go.kr/orginfoview.do?apba_id=' + apba_id

    res = requests.get(pub_real_url)
    time.sleep(2)
    
    if res.status_code == 200:
        print(f'processing {apba_id}')

        soup = BeautifulSoup(res.text, 'html.parser')
        rows = soup.select('#txt > table > tbody > tr')
        alio_dict = {}
        
        for r in rows:
            header = r.select_one('th').get_text()
            content = r.select_one('td').get_text()
            alio_dict[header]=content

        pub_contents.append(alio_dict)
    else:
        print(f'{apba_id}: {res.status_code}')

### User Agents

In [58]:
import requests

res = requests.get('https://naver.com')
res.request.headers

{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [59]:
import requests

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

res = requests.get('https://naver.com', headers=headers)
res.request.headers

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

# 동적 웹페이지 구조

![network image2](./img/network_2.jpg)

![network image3](./img/network_3.jpg)

# 예제 3: 서울복지포털

https://wis.seoul.go.kr/hope/customizedSearch.do

In [60]:
from selenium import webdriver
import pandas as pd
import time

In [61]:
driver = webdriver.Chrome('./chromedriver.exe')
url = 'https://wis.seoul.go.kr/hope/customizedSearch.do'
driver.get(url)

## 방식 1

In [63]:
# //*[@id="content"]/div[1]/div[2]/dl[1]/dd/div/div/table
# //*[@id="content"]/div[1]/div[2]/dl[2]/dd/div/div/table

center_info = driver.find_elements_by_tag_name('table')
print(len(center_info))
for c in center_info:
    print(c.text)

10
시설명 우리집 운영주체구분명 법인
시설유형명 장애인거주시설 시설종류명 장애인단기거주시설
법인명 대한예수교장로회연합회 법인대표자명 유영섭
전화번호 02-2658-7503 팩스번호 02-3665-9408
시설주소 서울특별시 강서구 양천로47길 70-6 홈페이지 주소 www.woorilove.org
이용(생활)
인원(실인원) 10 시소관부서 서울특별시 복지정책실 복지기획관 장애인복지정책과











## 방식 2

In [64]:
center_list = []
centers = driver.find_elements_by_xpath('//*[@id="content"]/div[1]/div[2]/dl')
for center in centers:
    drop_btn = center.find_element_by_xpath('dt/button')
    drop_btn.click()
    time.sleep(2)
    center_info = {}
    ths = center.find_elements_by_tag_name('th')
    tds = center.find_elements_by_tag_name('td')
    for n in range(len(ths)):
        center_info[ths[n].text]=tds[n].text
    center_list.append(center_info)

In [65]:
center_list

[{'시설명': '우리집',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설',
  '법인명': '대한예수교장로회연합회',
  '법인대표자명': '유영섭',
  '전화번호': '02-2658-7503',
  '팩스번호': '02-3665-9408',
  '시설주소': '서울특별시 강서구 양천로47길 70-6',
  '홈페이지 주소': 'www.woorilove.org',
  '이용(생활)\n인원(실인원)': '10',
  '시소관부서': '서울특별시 복지정책실 복지기획관 장애인복지정책과'},
 {'시설명': '선린원',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설',
  '법인명': '대한기독교총연합회',
  '법인대표자명': '이선',
  '전화번호': '02-2654-3372',
  '팩스번호': '02-2699-3372',
  '시설주소': '서울특별시 강서구 까치산로22길 35-5',
  '홈페이지 주소': 'www.선린원.kr',
  '이용(생활)\n인원(실인원)': '10',
  '시소관부서': '서울특별시 복지정책실 복지기획관 장애인복지정책과'},
 {'시설명': '강서희망의집',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설',
  '법인명': '대한예수교장로회연합회',
  '법인대표자명': '나건용',
  '전화번호': '02-2605-7980',
  '팩스번호': '02-2691-4682',
  '시설주소': '서울특별시 강서구 강서로34길 36',
  '홈페이지 주소': '',
  '이용(생활)\n인원(실인원)': '10',
  '시소관부서': '서울특별시 복지정책실 복지기획관 장애인복지정책과'},
 {'시설명': '헬렌의집장애인단기보호시설',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단

In [66]:
center_df = pd.DataFrame(center_list)
center_df

,시설명,운영주체구분명,시설유형명,시설종류명,법인명,법인대표자명,전화번호,팩스번호,시설주소,홈페이지 주소,이용(생활)\n인원(실인원),시소관부서
0,우리집,법인,장애인거주시설,장애인단기거주시설,대한예수교장로회연합회,유영섭,02-2658-7503,02-3665-9408,서울특별시 강서구 양천로47길 70-6,www.woorilove.org,10,서울특별시 복지정책실 복지기획관 장애인복지정책과
1,선린원,법인,장애인거주시설,장애인단기거주시설,대한기독교총연합회,이선,02-2654-3372,02-2699-3372,서울특별시 강서구 까치산로22길 35-5,www.선린원.kr,10,서울특별시 복지정책실 복지기획관 장애인복지정책과
2,강서희망의집,법인,장애인거주시설,장애인단기거주시설,대한예수교장로회연합회,나건용,02-2605-7980,02-2691-4682,서울특별시 강서구 강서로34길 36,,10,서울특별시 복지정책실 복지기획관 장애인복지정책과
3,헬렌의집장애인단기보호시설,법인,장애인거주시설,장애인단기거주시설,성프란치스꼬수녀회,조병희,02-830-8807,02-859-8807,서울특별시 구로구 구로중앙로27나길 18,,12,서울특별시 복지정책실 복지기획관 장애인복지정책과
4,꿈손단기거주시설,법인,장애인거주시설,장애인단기거주시설,서울가톨릭사회복지회,유경촌,02-904-0502,02-908-0501,서울특별시 강북구 인수봉로61길,,11,서울특별시 복지정책실 복지기획관 장애인복지정책과
5,송파치매케어센터,법인,노인의료복지시설,노인요양시설,청암노인복지재단,이성희,02-2203-9401,02-2203-9402,서울특별시 송파구 백제고분로32길 41,,28,서울특별시 복지정책실 복지기획관 어르신복지과
6,강남구립 논현요양센터,법인,노인의료복지시설,노인요양시설,남서울복지재단,화종부,02-541-0226,02-541-0116,서울특별시 강남구 강남대로128길 59,,29,서울특별시 복지정책실 복지기획관 어르신복지과
7,노원실버센터,법인,노인의료복지시설,노인요양시설,평화복지재단,이수남,02-978-7722,02-975-3524,서울특별시 노원구 동일로 1097,www.peacewelfare.or.kr,82,서울특별시 복지정책실 복지기획관 어르신복지과
8,상봉요양원,개인,노인의료복지시설,노인요양시설,상봉요양원,박서현,02-2207-8803,02-2209-8803,서울특별시 중랑구 용마산로 494,,23,서울특별시 복지정책실 복지기획관 어르신복지과
9,봄날서울요양원,개인,노인의료복지시설,노인요양시설,봄날서울요양원,이송,02-2661-8600,02-2661-8602,서울특별시 강서구 양천로1길 100,,121,서울특별시 복지정책실 복지기획관 어르신복지과


## 방식 3

In [70]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source, 'html.parser')
centers = soup.select('#content > div.sub_content > div.board_t1.no_out > dl')
center_list = []
for center in centers:
    center_info = {}
    ths = center.select('th')
    tds = center.select('td')
    for n in range(len(ths)):
        center_info[ths[n].get_text()]=tds[n].get_text().strip(' \t')
    center_list.append(center_info)

In [71]:
center_list

[{'시설명': '우리집',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설',
  '법인명': '대한예수교장로회연합회',
  '법인대표자명': '유영섭',
  '전화번호': '02-2658-7503',
  '팩스번호': '02-3665-9408',
  '시설주소': '서울특별시 강서구 양천로47길 70-6',
  '홈페이지 주소': 'www.woorilove.org',
  '이용(생활)인원(실인원)': '10',
  '시소관부서': '서울특별시 복지정책실 복지기획관 장애인복지정책과'},
 {'시설명': '선린원',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설',
  '법인명': '대한기독교총연합회',
  '법인대표자명': '이선',
  '전화번호': '02-2654-3372',
  '팩스번호': '02-2699-3372',
  '시설주소': '서울특별시 강서구 까치산로22길 35-5',
  '홈페이지 주소': 'www.선린원.kr',
  '이용(생활)인원(실인원)': '10',
  '시소관부서': '서울특별시 복지정책실 복지기획관 장애인복지정책과'},
 {'시설명': '강서희망의집',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설',
  '법인명': '대한예수교장로회연합회',
  '법인대표자명': '나건용',
  '전화번호': '02-2605-7980',
  '팩스번호': '02-2691-4682',
  '시설주소': '서울특별시 강서구 강서로34길 36',
  '홈페이지 주소': '',
  '이용(생활)인원(실인원)': '10',
  '시소관부서': '서울특별시 복지정책실 복지기획관 장애인복지정책과'},
 {'시설명': '헬렌의집장애인단기보호시설',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설'

In [72]:
center_df = pd.DataFrame(center_list)
center_df

,시설명,운영주체구분명,시설유형명,시설종류명,법인명,법인대표자명,전화번호,팩스번호,시설주소,홈페이지 주소,이용(생활)인원(실인원),시소관부서
0,우리집,법인,장애인거주시설,장애인단기거주시설,대한예수교장로회연합회,유영섭,02-2658-7503,02-3665-9408,서울특별시 강서구 양천로47길 70-6,www.woorilove.org,10,서울특별시 복지정책실 복지기획관 장애인복지정책과
1,선린원,법인,장애인거주시설,장애인단기거주시설,대한기독교총연합회,이선,02-2654-3372,02-2699-3372,서울특별시 강서구 까치산로22길 35-5,www.선린원.kr,10,서울특별시 복지정책실 복지기획관 장애인복지정책과
2,강서희망의집,법인,장애인거주시설,장애인단기거주시설,대한예수교장로회연합회,나건용,02-2605-7980,02-2691-4682,서울특별시 강서구 강서로34길 36,,10,서울특별시 복지정책실 복지기획관 장애인복지정책과
3,헬렌의집장애인단기보호시설,법인,장애인거주시설,장애인단기거주시설,성프란치스꼬수녀회,조병희,02-830-8807,02-859-8807,서울특별시 구로구 구로중앙로27나길 18,,12,서울특별시 복지정책실 복지기획관 장애인복지정책과
4,꿈손단기거주시설,법인,장애인거주시설,장애인단기거주시설,서울가톨릭사회복지회,유경촌,02-904-0502,02-908-0501,서울특별시 강북구 인수봉로61길,,11,서울특별시 복지정책실 복지기획관 장애인복지정책과
5,송파치매케어센터,법인,노인의료복지시설,노인요양시설,청암노인복지재단,이성희,02-2203-9401,02-2203-9402,서울특별시 송파구 백제고분로32길 41,,28,서울특별시 복지정책실 복지기획관 어르신복지과
6,강남구립 논현요양센터,법인,노인의료복지시설,노인요양시설,남서울복지재단,화종부,02-541-0226,02-541-0116,서울특별시 강남구 강남대로128길 59,,29,서울특별시 복지정책실 복지기획관 어르신복지과
7,노원실버센터,법인,노인의료복지시설,노인요양시설,평화복지재단,이수남,02-978-7722,02-975-3524,서울특별시 노원구 동일로 1097,www.peacewelfare.or.kr,82,서울특별시 복지정책실 복지기획관 어르신복지과
8,상봉요양원,개인,노인의료복지시설,노인요양시설,상봉요양원,박서현,02-2207-8803,02-2209-8803,서울특별시 중랑구 용마산로 494,,23,서울특별시 복지정책실 복지기획관 어르신복지과
9,봄날서울요양원,개인,노인의료복지시설,노인요양시설,봄날서울요양원,이송,02-2661-8600,02-2661-8602,서울특별시 강서구 양천로1길 100,,121,서울특별시 복지정책실 복지기획관 어르신복지과


## Pagination

In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [11]:
driver = webdriver.Chrome('./chromedriver.exe')
url = 'https://wis.seoul.go.kr/hope/customizedSearch.do'
driver.get(url)

In [73]:
current_page_n = 1
center_list = []
while current_page_n < 3:
    driver.execute_script(f'pagedIng({current_page_n});')
    print(f'processing page {current_page_n}')
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    centers = soup.select('#content > div.sub_content > div.board_t1.no_out > dl')
    
    for center in centers:
        center_info = {}
        ths = center.select('th')
        tds = center.select('td')
        for n in range(len(ths)):
            center_info[ths[n].get_text()]=tds[n].get_text().strip(' \t')
        center_list.append(center_info)
        
    current_page_n += 1

processing page 1
processing page 2


In [74]:
center_list

[{'시설명': '우리집',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설',
  '법인명': '대한예수교장로회연합회',
  '법인대표자명': '유영섭',
  '전화번호': '02-2658-7503',
  '팩스번호': '02-3665-9408',
  '시설주소': '서울특별시 강서구 양천로47길 70-6',
  '홈페이지 주소': 'www.woorilove.org',
  '이용(생활)인원(실인원)': '10',
  '시소관부서': '서울특별시 복지정책실 복지기획관 장애인복지정책과'},
 {'시설명': '선린원',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설',
  '법인명': '대한기독교총연합회',
  '법인대표자명': '이선',
  '전화번호': '02-2654-3372',
  '팩스번호': '02-2699-3372',
  '시설주소': '서울특별시 강서구 까치산로22길 35-5',
  '홈페이지 주소': 'www.선린원.kr',
  '이용(생활)인원(실인원)': '10',
  '시소관부서': '서울특별시 복지정책실 복지기획관 장애인복지정책과'},
 {'시설명': '강서희망의집',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설',
  '법인명': '대한예수교장로회연합회',
  '법인대표자명': '나건용',
  '전화번호': '02-2605-7980',
  '팩스번호': '02-2691-4682',
  '시설주소': '서울특별시 강서구 강서로34길 36',
  '홈페이지 주소': '',
  '이용(생활)인원(실인원)': '10',
  '시소관부서': '서울특별시 복지정책실 복지기획관 장애인복지정책과'},
 {'시설명': '헬렌의집장애인단기보호시설',
  '운영주체구분명': '법인',
  '시설유형명': '장애인거주시설',
  '시설종류명': '장애인단기거주시설'

In [75]:
center_df = pd.DataFrame(center_list)
center_df

,시설명,운영주체구분명,시설유형명,시설종류명,법인명,법인대표자명,전화번호,팩스번호,시설주소,홈페이지 주소,이용(생활)인원(실인원),시소관부서
0,우리집,법인,장애인거주시설,장애인단기거주시설,대한예수교장로회연합회,유영섭,02-2658-7503,02-3665-9408,서울특별시 강서구 양천로47길 70-6,www.woorilove.org,10,서울특별시 복지정책실 복지기획관 장애인복지정책과
1,선린원,법인,장애인거주시설,장애인단기거주시설,대한기독교총연합회,이선,02-2654-3372,02-2699-3372,서울특별시 강서구 까치산로22길 35-5,www.선린원.kr,10,서울특별시 복지정책실 복지기획관 장애인복지정책과
2,강서희망의집,법인,장애인거주시설,장애인단기거주시설,대한예수교장로회연합회,나건용,02-2605-7980,02-2691-4682,서울특별시 강서구 강서로34길 36,,10,서울특별시 복지정책실 복지기획관 장애인복지정책과
3,헬렌의집장애인단기보호시설,법인,장애인거주시설,장애인단기거주시설,성프란치스꼬수녀회,조병희,02-830-8807,02-859-8807,서울특별시 구로구 구로중앙로27나길 18,,12,서울특별시 복지정책실 복지기획관 장애인복지정책과
4,꿈손단기거주시설,법인,장애인거주시설,장애인단기거주시설,서울가톨릭사회복지회,유경촌,02-904-0502,02-908-0501,서울특별시 강북구 인수봉로61길,,11,서울특별시 복지정책실 복지기획관 장애인복지정책과
5,송파치매케어센터,법인,노인의료복지시설,노인요양시설,청암노인복지재단,이성희,02-2203-9401,02-2203-9402,서울특별시 송파구 백제고분로32길 41,,28,서울특별시 복지정책실 복지기획관 어르신복지과
6,강남구립 논현요양센터,법인,노인의료복지시설,노인요양시설,남서울복지재단,화종부,02-541-0226,02-541-0116,서울특별시 강남구 강남대로128길 59,,29,서울특별시 복지정책실 복지기획관 어르신복지과
7,노원실버센터,법인,노인의료복지시설,노인요양시설,평화복지재단,이수남,02-978-7722,02-975-3524,서울특별시 노원구 동일로 1097,www.peacewelfare.or.kr,82,서울특별시 복지정책실 복지기획관 어르신복지과
8,상봉요양원,개인,노인의료복지시설,노인요양시설,상봉요양원,박서현,02-2207-8803,02-2209-8803,서울특별시 중랑구 용마산로 494,,23,서울특별시 복지정책실 복지기획관 어르신복지과
9,봄날서울요양원,개인,노인의료복지시설,노인요양시설,봄날서울요양원,이송,02-2661-8600,02-2661-8602,서울특별시 강서구 양천로1길 100,,121,서울특별시 복지정책실 복지기획관 어르신복지과


### csv

In [14]:
import csv


file_name = "서울시복지시설.csv"
f = open(file_name, 'w', encoding='utf-8-sig', newline='')
writer = csv.writer(f)

current_page_n = 1
while current_page_n < 3:
    driver.execute_script(f'pagedIng({current_page_n});')
    print(f'processing page {current_page_n}')
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    centers = soup.select('#content > div.sub_content > div.board_t1.no_out > dl')
    for center in centers:
        tds = center.select('td')
        content = [td.text for td in tds]
    current_page_n += 1
    
f.close()

processing page 1
processing page 2


# 비교: 나라일터

https://www.gojobs.go.kr/frameMenu.do?url=apmList.do%3FsearchJobsecode%3D050%26searchEmpmnsecode%3De05&menuNo=47&mngrMenuYn=N&message=

In [ ]:
# https://www.gojobs.go.kr/apmList.do?searchJobsecode=050&amp;searchEmpmnsecode=e05&menuNo=47&empmnsn=0&searchJobsecode=&searchBbssecode=0&searchBbssn=0
# https://www.gojobs.go.kr/apmList.do?searchJobsecode=050&amp;searchEmpmnsecode=e05&menuNo=47&empmnsn=0&searchJobsecode=&searchBbssecode=0&searchBbssn=0&pageIndex=2
# https://www.gojobs.go.kr/apmView.do?empmnsn=135922&searchJobsecode=050

# 연습: Daum 댓글

In [76]:
from selenium import webdriver
import pandas as pd
import time

driver = webdriver.Chrome('./chromedriver.exe')

In [78]:
driver.get('https://news.v.daum.net/v/20210811213002144')
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [79]:
chrono_btn = driver.find_element_by_xpath('//*[@id="alex-area"]/div/div/div/div[3]/ul[1]/li[4]/button')
chrono_btn.click()

In [80]:
click_count = 0
while True:
    try:
        more_btn = driver.find_element_by_xpath('//*[@id="alex-area"]/div/div/div/div[3]/div[3]')
        more_btn.click()
        
        time.sleep(5)
        click_count += 1
        
        if click_count == 3:
            break
    except:
        break

In [81]:
cmts = driver.find_elements_by_css_selector('div.cmt_box > ul.list_comment > li')

In [82]:
len(cmts)

33

In [ ]:
div > strong > span > a > span:nth-child(2)

In [83]:
cmt_list = []
for cmt in cmts:
    cmt_list.append({
        'cmt_name': cmt.find_element_by_css_selector('div > strong > span > a').text,
        'cmt_body': cmt.find_element_by_css_selector('div > p').text,
        'cmt_like': cmt.find_element_by_css_selector('button.btn_g.btn_recomm.\#like > span.num_txt').text,
        'cmt_dislike': cmt.find_element_by_css_selector('button.btn_g.btn_oppose.\#dislike > span.num_txt').text
    })

In [84]:
cmt_list

[{'cmt_name': 'Ggoirt',
  'cmt_body': '인간종족때문에 애먼 동물들이 가장 먼저 희생되는 게 안타까울 뿐이다.',
  'cmt_like': '199',
  'cmt_dislike': '12'},
 {'cmt_name': 'sun', 'cmt_body': '지구야…', 'cmt_like': '50', 'cmt_dislike': '7'},
 {'cmt_name': '동백',
  'cmt_body': '표현도 참\n지옥이 있냐\n헤드라이 제목 자체가 지옥이네',
  'cmt_like': '0',
  'cmt_dislike': '8'},
 {'cmt_name': 'sungyun',
  'cmt_body': '.......인간 벌받나봐',
  'cmt_like': '119',
  'cmt_dislike': '3'},
 {'cmt_name': '균',
  'cmt_body': '인간멸종 지구멸망',
  'cmt_like': '43',
  'cmt_dislike': '8'},
 {'cmt_name': '2021년',
  'cmt_body': '자연이 파괴되든 샤람이 죽어나가든 돈은 돌고 주가는 최고가 행진인 세상에 누가 그런거 신경쓰나^^  죽은 놈만 바보지',
  'cmt_like': '1',
  'cmt_dislike': '1'},
 {'cmt_name': '.',
  'cmt_body': '내가 살고 있는 곳도 아닌데 사진 보니까 뭔가 눈물난다 ㅠㅠ 무섭고 안됫고......',
  'cmt_like': '200',
  'cmt_dislike': '8'},
 {'cmt_name': '그 무엇 보다',
  'cmt_body': '화석연료와 원전 사용을 대폭 줄여야 합니다. 그리고 대체 에너지를 개발합니다.  또한 아나바다 운동을 생활화합니다. 아껴쓰고 나눠쓰고 바꿔쓰고 다시 쓰는 운동을 적극적으로 전개해야 합니다.',
  'cmt_like': '260',
  'cmt_dislike': '24'},
 {'cmt_name': '조성얼라',
  '

In [85]:
cmt_df = pd.DataFrame(cmt_list)

In [86]:
cmt_df

,cmt_name,cmt_body,cmt_like,cmt_dislike
0,Ggoirt,인간종족때문에 애먼 동물들이 가장 먼저 희생되는 게 안타까울 뿐이다.,199,12
1,sun,지구야…,50,7
2,동백,표현도 참\n지옥이 있냐\n헤드라이 제목 자체가 지옥이네,0,8
3,sungyun,.......인간 벌받나봐,119,3
4,균,인간멸종 지구멸망,43,8
5,2021년,자연이 파괴되든 샤람이 죽어나가든 돈은 돌고 주가는 최고가 행진인 세상에 누가 그런...,1,1
6,.,내가 살고 있는 곳도 아닌데 사진 보니까 뭔가 눈물난다 ㅠㅠ 무섭고 안됫고......,200,8
7,그 무엇 보다,화석연료와 원전 사용을 대폭 줄여야 합니다. 그리고 대체 에너지를 개발합니다. 또...,260,24
8,조성얼라,디지는 사람만 디지지 아작도 걱정조차 안하고 지구 종말을 향하여 여전히 돈벌이에 급...,132,5
9,Kevin,개념 훈련이 되지 않고 그 동안 막살아온 댓가.\n개개인이 윤리와 도덕을 배웠으면 ...,179,6
